# Deep Residual Network

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [2]:
# Device Configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
device

device(type='cuda')

In [16]:
# Hyper-parameters
num_epochs = 20
batch_size = 100
learning_rate = 0.001

In [5]:
# image preprocessing modules
transform = transforms.Compose([
                                transforms.Pad(4),
                                transforms.RandomHorizontalFlip(),
                                transforms.RandomCrop(32),
                                transforms.ToTensor()
])

In [6]:
# CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                            train=True,
                                            transform=transform,
                                            download=True)

test_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                            train=False,
                                            transform=transforms.ToTensor())


Extracting ../../data/cifar-10-python.tar.gz to ../../data/


In [7]:
# Data Loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size,
                                          shuffle=False)

In [8]:
# 3x3 convolution
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)

In [9]:
# Residual block
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__() # calls for cooperative multiple inheritance
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample
    
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x) # to match the channel numbers
        out += residual
        out = self.relu(out)
        return out

In [10]:
# ResNet
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 16
        self.conv = conv3x3(3, 16)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 16, layers[0])
        self.layer2 = self.make_layer(block, 32, layers[1], 2)
        self.layer3 = self.make_layer(block, 64, layers[2], 2)
        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64, num_classes)
    
    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(
                conv3x3(self.in_channels, out_channels, stride=stride),
                nn.BatchNorm2d(out_channels)
            )
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels)) # add block n times
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [11]:
model = ResNet(ResidualBlock, [2, 2, 2]).to(device)

In [12]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [13]:
# For updating learning rate
def update_lr(optimizer, lr):
    for param_group in optimizer.param_groups:
        param_group['lr']= lr

In [14]:
optimizer.param_groups[0]

{'amsgrad': False,
 'betas': (0.9, 0.999),
 'eps': 1e-08,
 'lr': 0.001,
 'params': [Parameter containing:
  tensor([[[[-0.1467,  0.0985,  0.1767],
            [ 0.1724,  0.0855, -0.0161],
            [ 0.1651,  0.0066, -0.0711]],
  
           [[ 0.1405, -0.1772,  0.1187],
            [ 0.0427,  0.1103,  0.1152],
            [-0.0558,  0.0934,  0.0011]],
  
           [[ 0.1868, -0.1339,  0.1237],
            [-0.0161, -0.0111, -0.1887],
            [-0.1904,  0.0076, -0.0072]]],
  
  
          [[[ 0.0025,  0.0823, -0.1463],
            [ 0.1872, -0.1873,  0.0078],
            [-0.0262,  0.1718, -0.1585]],
  
           [[ 0.0824,  0.1476, -0.1040],
            [-0.0675, -0.1099, -0.0974],
            [-0.0706, -0.1041,  0.0149]],
  
           [[-0.0418,  0.0775, -0.0425],
            [-0.1470,  0.1552, -0.0544],
            [-0.0976, -0.0899, -0.0266]]],
  
  
          [[[-0.0152,  0.0136, -0.0194],
            [-0.1738,  0.1444, -0.0370],
            [ 0.0507,  0.1439, -0.1565]],


In [17]:
# Train the model
total_step = len(train_loader)
curr_lr = learning_rate
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        #Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f}".format(
                epoch+1, num_epochs, i+1, total_step, loss.item()
            ))
    
    # Decay learning rate
    if (epoch+1) & 20 == 0:
        curr_lr /= 3
        update_lr(optimizer, curr_lr)


Epoch [1/20], Step [100/500] Loss: 1.0936
Epoch [1/20], Step [200/500] Loss: 0.9938
Epoch [1/20], Step [300/500] Loss: 0.8897
Epoch [1/20], Step [400/500] Loss: 0.8516
Epoch [1/20], Step [500/500] Loss: 0.9895
Epoch [2/20], Step [100/500] Loss: 0.8284
Epoch [2/20], Step [200/500] Loss: 1.1095
Epoch [2/20], Step [300/500] Loss: 0.8200
Epoch [2/20], Step [400/500] Loss: 0.8105
Epoch [2/20], Step [500/500] Loss: 0.7644
Epoch [3/20], Step [100/500] Loss: 0.7919
Epoch [3/20], Step [200/500] Loss: 0.8714
Epoch [3/20], Step [300/500] Loss: 0.7512
Epoch [3/20], Step [400/500] Loss: 0.6293
Epoch [3/20], Step [500/500] Loss: 0.6657
Epoch [4/20], Step [100/500] Loss: 0.7039
Epoch [4/20], Step [200/500] Loss: 0.6691
Epoch [4/20], Step [300/500] Loss: 0.7290
Epoch [4/20], Step [400/500] Loss: 0.5972
Epoch [4/20], Step [500/500] Loss: 0.8428
Epoch [5/20], Step [100/500] Loss: 0.7885
Epoch [5/20], Step [200/500] Loss: 0.5976
Epoch [5/20], Step [300/500] Loss: 0.5750
Epoch [5/20], Step [400/500] Loss:

In [19]:
# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))

# save the model
torch.save(model.state_dict(), 'resnet.ckpt')

Accuracy of the model on the test images: 76.26 %
